In [1]:
import os

In [2]:
%pwd

'c:\\Users\\jhasu\\OneDrive\\Desktop\\chickenclassifier\\chickendisease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
from pathlib import Path

In [5]:
from dataclasses import dataclass


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    Unzip_dir:Path



In [6]:
from src.cnnClassifier.constants import *

In [7]:
from src.cnnClassifier.utils.common import read_yaml,create_directory


In [8]:
class ConfigurationManager:
    def __init__(self,
     config_filepath=CONFIG_FILE_PATH,
     param_filepath=PARAM_FILE_PATH):
     self.config = read_yaml(config_filepath)
     self.param = read_yaml(param_filepath)

     create_directory([self.config.artifacts_root])

    def get_data_ingestion_configuration(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directory([config.root_dir])
        data_ingestion= DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            Unzip_dir=config.Unzip_zar)

        return data_ingestion


In [9]:
import os
from urllib.request import urlretrieve
from  src.cnnClassifier.utils.common import get_sizes
from  src.cnnClassifier.loggs import  logger
import zipfile

In [10]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_data(self):
        path= Path(self.config.local_data_file)
        if not path.exists():
            filename,headers= urlretrieve(
                url= self.config.source_URL,
                filename=str(path)
                )
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        else :
            logger.info(f"file already exists of size : {get_sizes(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.Unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Zip file extracted to : {unzip_path}")

In [11]:
try:
    config= ConfigurationManager()
    data_config=  config.get_data_ingestion_configuration()
    data_ingestion= DataIngestion(data_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-23 16:22:45,048: INFO : common : yaml file: config\config.yaml loaded successfully ]
[2024-12-23 16:22:45,050: INFO : common : yaml file: params.yaml loaded successfully ]
[2024-12-23 16:22:45,050: INFO : common : Directory: artifacts created successfully ]
[2024-12-23 16:22:45,054: INFO : common : Directory: artifacts/data_ingestion created successfully ]
[2024-12-23 16:22:59,060: INFO : 1144568391 : artifacts\data_ingestion\data.zip downloaded with the following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7302:5BFCD:8BCFC:C6B9A:676940FB
Accept-Ranges: bytes
Date: Mon, 23 Dec 2024 10:52:45 GMT
Via: 1.1 varnish
X-Ser